In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import networkx as nx
from sklearn.neighbors import kneighbors_graph

In [3]:
data=pd.read_csv('/home/mandana/Data/slsa-surfguard-NEW_MG3_final1_wave_wind_current_rename_cgp.csv')

In [ ]:
#GoaldCoast
CROP_LON_MIN, CROP_LON_MAX =153.4, 154.87

CROP_LAT_MIN, CROP_LAT_MAX =-29.2,-27.753

In [ ]:
# CROP_LON_MIN, CROP_LON_MAX = 151.341, 152.579
# CROP_LAT_MIN, CROP_LAT_MAX = -34.015, -32.863

In [9]:
data_goaldcoast = data[(data['lon'] >= CROP_LON_MIN) & (data['lon'] <= CROP_LON_MAX) &
                       (data['lat'] >= CROP_LAT_MIN) & (data['lat'] <= CROP_LAT_MAX)]

In [12]:
temp1=data_goaldcoast.groupby('time').count()

In [15]:
temp2=data_goaldcoast.groupby('time').sum()

In [17]:
temp=pd.merge(temp1, temp2, on='time', suffixes=('_count', '_sum'))

In [24]:
temp.reset_index(inplace=True)

In [28]:
temp[['time','beach_key_lower_count','stings_sum']].to_csv('/home/mandana/Data/Data5925T3G4/goaldcoast_stings.csv', index=False)

In [29]:
temp0=pd.read_csv('/home/mandana/Data/Data5925T3G4/goaldcoast_stings.csv')

In [ ]:
df3=pd.read_csv('/home/mandana/Data/slsa-surfguard-NEW_MG3_final1_wave_wind_current_rename_cgp_merged_wind.csv')
df3['time'] = pd.to_datetime(df3['time'])
# df3['beach_key_lower']= df3['beach_key_'].str.lower()
df3=pd.merge(df3, dff[['embaymentisation', 'orientation','time', 'beach_key_lower']], on=['beach_key_lower','time'], how='left',)

df3['stings_binary'] = 0
df3.loc[df3['stings'] != 0, 'stings_binary'] = 1


df3['day_of_year'] = df3['time'].dt.dayofyear 
df3['is_weekend'] = df3['time'].dt.weekday >= 5  
df3 = df3[df3['time'].dt.month.isin([1,2,3,4,8,9,10,11])].copy() # Filter for all months

df3 = df3.sort_values(['beach_key_lower', 'time'])
df3=df3[df3['state'] == 'QLD'].copy()
df3=df3[df3['stings']!=1].copy()  # Exclude rows where stings is 1

In [2]:
# Reload the processed dataset
file_path = '/home/mandana/Data/slsa-surfguard-NEW_MG3_final1_wave_wind_current_rename_cgp_merged_wind.csv'

df = pd.read_csv(file_path)
df['time'] = pd.to_datetime(df['time'])
df = df.sort_values(['beach_key_lower', 'time'])

target = "stings_binary"


In [3]:
df=df[(df['stings']>1)| (df['stings']==0)].copy() 
df = df[df['species'] == 'Physalia (Bluebottle)'].copy()
df=df[df['state'] == 'QLD'].copy() 



df = df[df['time'].dt.month.isin([1])].copy() 

df['day_of_year'] = df['time'].dt.dayofyear # 3% more, becasue i included only jan
df['is_weekend'] = df['time'].dt.weekday >= 5   #no diff

features = [ 'ts_wind', 'uas_wind','vas_wind', 'uas','vas','ts',
             'temp',
            'is_weekend','lat','lon','embaymentisation', 'orientation', 'day_of_year',]


In [ ]:
df3.dropna(subset=features + [target], inplace=True)
df3.reset_index(drop=True, inplace=True)